# Análisis de Microbiomas en R

Este tutorial ofrece una visión integral de las principales funciones de la librería [microeco](https://github.com/ChiLiubio/microeco). Este paquete de R está diseñado para asistir al usuario en el análisis de datos de microbiomas utilizando un rango amplio de metodologías que facilitan la experiencia del análisis.  

A lo largo del tutorial, aprenderá a cargar datos en R, manipularlos y normalizarlos, calcular abundancias relativas, realizar análisis descriptivos basados en dichas abundancias, llevar a cabo análisis de diversidad (índices alfa y beta) y representar los resultados de forma gráfica.

## 1. Instalación de paquetes 

Antes de proceder con la instalación, es importante señalar que, al igual que muchas otras librerías de R, **microeco** es un paquete que se compone por diversas dependencias. Estas dependencias no son más que otras librerías de R que deben instalarse previamente para poder utilizar microeco de una forma adecuada.

El comando `install.packages("BiocManager")` en R se utiliza para instalar el paquete [BiocManager](https://www.bioconductor.org/install/) desde CRAN. Este paquete es una herramienta esencial para trabajar con paquetes del proyecto [Bioconductor](https://www.bioconductor.org/), que proporciona herramientas bioinformáticas para el análisis y comprensión de datos biológicos. Bioconductor es un repositorio especializado en herramientas bioinformáticas y estadísticas diseñadas para el análisis de datos genómicos, transcriptómicos, proteómicos, y más.

In [ ]:
# Instalar BiocManager

install.packages("BiocManager")

In [ ]:
# Instalar paquetes con BiocManager

BiocManager::install("ggtree")
BiocManager::install("metagenomeSeq")
BiocManager::install("ALDEx2")
BiocManager::install("ANCOMBC")
install.packages("file2meco", repos = BiocManager::repositories())
install.packages("MicrobiomeStat", repos = BiocManager::repositories())
install.packages("WGCNA", repos = BiocManager::repositories())
install.packages("remotes")
install.packages("devtools")

In [ ]:
# Instalar paquetes

install.packages("file2meco", repos = BiocManager::repositories())
install.packages("MicrobiomeStat", repos = BiocManager::repositories())
install.packages("WGCNA", repos = BiocManager::repositories())
install.packages("remotes")
install.packages("devtools")
remotes::install_github("jbisanz/qiime2R")

> **Nota:** Es común que las librerías de R sufran actualizaciones constantemente, por lo tanto, a la hora accesar esta guía podrían existir cambios que no estén contemplados en la misma. Se recomienda visitar la siguiente página en caso de que ocurra algún inconveniente https://chiliubio.github.io/microeco_tutorial/ así como https://github.com/ChiLiubio/microeco para la instalación de la librería.

Finalmente se realiza la instalación del paquete *microeco*

In [ ]:
# Instalar microeco paquetes directamente desde un repositorio de GitHub

devtools::install_github("ChiLiubio/microeco")

Una vez descargados los paquetes o librerías, iniciaremos las librerías utilizando la función `library()`

In [ ]:
# Cargar librerías

library(microeco)
library(qiime2R)
library(ggplot2)
library(magrittr)
library(RColorBrewer)
library(ggalluvial)

## 2. Descargar datos

Los datos utilizados en este tutorial se encuentra en un repositorio en Zenodo: https://zenodo.org/records/13972776.

Para descargarlos, utilizaremos la función de R `download.file()`

In [ ]:
urls <- c(
  "https://zenodo.org/records/13972776/files/classification.qza",
  "https://zenodo.org/records/13972776/files/sample-metadata.tsv",
  "https://zenodo.org/records/13972776/files/table.qza",
  "https://zenodo.org/records/13972776/files/rooted_tree.qza"
)

# Nombrar archivos
destfiles <- c("classification.qza", "sample-metadata.tsv", "table.qza", "rooted_tree.qza")

# Descargar los archivos
for (i in seq_along(urls)) {
  download.file(url = urls[i], destfile = destfiles[i], mode = "wb")
  cat(sprintf("Archivo %s descargado correctamente.\n", destfiles[i]))
}

Obtendrá el siguiente resultado:

![descarga](./Images/descarga.png)

## 3. Cargar datos

Para cargar los datos, utilizaremos una librería implementada en R para utilizar los archivos producidos por el programa [QIIME2](https://qiime2.org/). Esta librería lleva el nombre de **qiime2R** y ya fue descargada y cargada en los pasos previos. 

Seguidamente se debe utilizar la función `setwd()` para seleccionar el directorio donde se encuentran los archivos de datos, los cuales están disponibles en la carpeta del tutorial.

In [ ]:
# Seleccionar el directorio

setwd("/content")

In [ ]:
# Cargar datos

datos= qza_to_phyloseq(features = "table.qza",
  taxonomy =   "classification.qza",
  tree = "rooted_tree.qza",
  metadata = "sample-metadata.tsv")

Extraemos cada uno de los archivos de datos para implementarlos en las funciones de microeco.

In [ ]:
otu_table = as.data.frame(datos@otu_table)
metadatos = data.frame(datos@sam_data)
arbol = datos@phy_tree
taxonomia = data.frame(datos@tax_table)

Utilizando la función `microtable$new` se puede generar un nuevo conjunto de datos con los archivos cargados. La diferencia principal radica en que este objeto sí es apto para ser utilizado con las demás funciones de la librería.

In [ ]:
# Archivo de datos microtable (microeco)
dataset <-
  microtable$new(
    otu_table = otu_table,
    sample_table = metadatos,
    phylo_tree = arbol,
    tax_table = taxonomia

## 4. Rarefacción

Un término común en el análisis de datos de microbiomas es el de *rarefacción*, este implica un proceso de muestreo aleatorio sobre las unidades contenidas en los OTUS en el cual se debe definir un valor para realizar el muestreo y estandarizar las unidades. Usualmente el valor seleccionado es el de la muestra que contiene la menor cantidad de secuencias. Para verificar el rango de secuencias de los OTUS se utiliza la primera línea de código, la cual indica que la muestra con el menor número de secuencias contiene 1067, por este motivo se ejecuta la rarefacción fijando este valor y utilizando la función `rarefy_samples`. Al ser un muestreo aleatorio se debe fijar una semilla para poder replicar los resultados.

In [ ]:
# Fijamos una semilla para replicar los resultados

set.seed(1)

In [ ]:
# Verificación del rango de valores de los OTU

dataset$sample_sums() %>% range

In [ ]:
# Rarefacción

dataset$rarefy_samples(sample.size = 1067)

> **Nota:** La aplicación de la técnica de rarefacción ha sido custionada a lo largo de los años en el estudio de microbiomas. Para comprender más a profundidad sobre esta discusión se recomienda la siguiente lectura: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003531

## 4. Cálculo de abundancia relativa

Una de las características más importantes de los análisis de microbiomas es identificar las principales bacterias que están presentes en las unidades de estudio, para esto se debe utilizar la abundancia relativa, la cual indica la presencia en una escala de 0 a 1 de cada bacteria en cada unidad de estudio.
La siguiente función permite calcular la abundancia en los distintos niveles de la filogenia. Esta abundancia se podrá acceder en `dataset$taxa_abund`. Agregando otro `$` después de `taxa_abund` se puede accesar a la abundancia en diversos niveles taxonómicos como se puede ver en la segunda línea de código.

In [ ]:
# Cálculo de abundancia

dataset$cal_abund()

In [ ]:
# Acceso a los datos
# Se utiliza [1:2,1:2] para mostrar las 2 primeras ranas y bacterias

dataset$taxa_abund$Family[1:2,1:2]

## 5. Funciones de manipulación de datos

### 5.1 Agrupación de taxonomía

En tutoriales anteriores los OTUS se podían agrupar en diversos niveles de la filogenia, sin embargo, había que utilizar diversas librerías para lograr esto. La librería *microtable* permite segregar los OTUS mediante la función `merge_taxa`, esto hará que la información contenida dentro de la tabla de OTUS se ajuste según el nivel de taxonomía (taxa) deseado. Por ejemplo: Kingdom, Phylum, Class, Order, Family o Genus

In [ ]:
gen = dataset$merge_taxa(taxa = "Genus")
gen

Recuerde que la cantidad de OTUS al estar agrupada se reduce con respecto a los presentes en dataset.

### 5.2 Clonación y modificación de datos

En ocasiones se necesitan realizar transformaciones dentro de un archivo de datos, como la que se hizo cuando se creó el objeto gen, sin embargo, siempre es importante contar con el archivo de datos original para mantener un orden a la hora de programar.
Para esto *microeco* implementa la función clone. En el siguiente ejemplo primero se clona el set de datos y después se filtra por una columna de los metadatos, mientras que en el segundo ejemplo se filtra por una bacteria en específico del conjunto de datos de taxonomía.